# Boltzmann portfolios

### Part 2: Parametization and dynamics of weights

We develop an alternative to the traditional mean-variance
framework ("*Markowitz*" portfolios) called ***Boltzmann*** portfolios
which handle uncertainty from the standpoint of cross-entropy
and optimal sequential decisions.
The improved result is a faster online algorithm which is more robust.

Markowitz portfolios are optimal in the arithmetic mean-variance
framework for a *single-period*. They are fragile to changing
market conditions, much like elegant battle strategies which
crumble under harsh war conditions.
In contrast, Boltzmann portfolios are designed to be
adaptive over multiple periods to maximize final wealth.
Techniques have been borrowed from Bayesian and
reinforcement learning.

Here in Part 2, we explore variations in parametization and weights over time.
Sequential decisions based on such information will be covered in Part 3
(where data windowing will be replaced by online filtration).
Part 4 will cover the mathematical proof of concept, but here
are many numerical lessons which will help guide us.

Appendix 1 herein examines in detail how the `boltzportfolio()`
algorithm robustly handled leptokurtotic events
from the global credit crisis of August 2011.

*Dependencies:*

- Repository: https://github.com/rsvp/fecon235
     
*CHANGE LOG*

    2017-07-21  Edit per stable Part 1. Add Appendix 1.
    2017-06-29  Investigate temperature parametization, weight dynamics.
    2017-06-28  Functions moved to ys_prtf_boltzmann module.
    2017-06-27  First version, functions numerically tested.

In [2]:
from fecon235.fecon235 import *

In [3]:
#  PREAMBLE-p6.15.1223d :: Settings and system details
from __future__ import absolute_import, print_function, division
system.specs()
pwd = system.getpwd()   # present working directory as variable.
print(" ::  $pwd:", pwd)
#  If a module is modified, automatically reload it:
%load_ext autoreload
%autoreload 2
#       Use 0 to disable this feature.

#  Notebook DISPLAY options:
#      Represent pandas DataFrames as text; not HTML representation:
import pandas as pd
pd.set_option( 'display.notebook_repr_html', False )
from IPython.display import HTML # useful for snippets
#  e.g. HTML('<iframe src=http://en.mobile.wikipedia.org/?useformat=mobile width=700 height=350></iframe>')
from IPython.display import Image 
#  e.g. Image(filename='holt-winters-equations.png', embed=True) # url= also works
from IPython.display import YouTubeVideo
#  e.g. YouTubeVideo('1j_HxD4iLn8', start='43', width=600, height=400)
from IPython.core import page
get_ipython().set_hook('show_in_pager', page.as_hook(page.display_page), 0)
#  Or equivalently in config file: "InteractiveShell.display_page = True", 
#  which will display results in secondary notebook pager frame in a cell.

#  Generate PLOTS inside notebook, "inline" generates static png:
%matplotlib inline   
#          "notebook" argument allows interactive zoom and resize.

 ::  Python 2.7.13
 ::  IPython 5.1.0
 ::  jupyter_core 4.2.1
 ::  notebook 4.1.0
 ::  matplotlib 1.5.1
 ::  numpy 1.11.0
 ::  scipy 0.17.0
 ::  sympy 1.0
 ::  pandas 0.19.2
 ::  pandas_datareader 0.2.1
 ::  Repository: fecon235 v5.17.0603 devPrtf
 ::  Timestamp: 2017-07-21T21:57:22Z
 ::  $pwd: /media/yaya/virt15h/virt/dbx/Dropbox/ipy/fecon235/nb


## Introduction

***It is important to first understand Part 1 of this series,
and the narrative behind the code.
We dive in with its condensed content, after constructing a dataframe.***

Full details are in the module, `lib/ys_prtf_boltzmann.py`.
Information on any function **foo()** can be displayed in a cell
by `foo?` or with more details by `foo??`.

## Download data and construct a dataframe

We retrieve the following data of daily frequency
representing equities worldwide and gold by five ETF securities: 

In [4]:
#  Convenient dictionary set in fecon235.py,
#  where key is world region, and value is its fecon235 data code:
world4d

{'America': 's4spy',
 'Emerging': 's4eem',
 'Europe': 's4ezu',
 'Gold': 's4gld',
 'Japan': 's4ewj'}

In [5]:
#  Or manually specify your own dictionary here:
prices_dic = world4d

In [6]:
#  Download data into a dataframe, alphabetically by key:
prices = groupget( prices_dic, maxi=3650 )
#  ... about ten years worth.

 ::  Retrieved from Google Finance: SPY
 ::  Retrieved from Google Finance: EEM
 ::  Retrieved from Google Finance: EZU
 ::  Retrieved from Google Finance: GLD
 ::  Retrieved from Google Finance: EWJ


In [7]:
#  Define start and end for prices, when applicable:
start = '2011-01-01'
end = '2017-06-26'

## Visual summary of Part 1


```
    prices ---> cov ---> globalw
      |                    |
      |                  trimit  <-- floor
      |                  renormalize
      |                    |
      v                    v
      |                    |
    gemrat              weights
      |                    |
      |________scores______|
                 |
                 |                   Boltzmann
      temp --> softmax --> probs --> pweights

             
```

where the function **boltzportfolio()** unifies the supporting functions.

In [8]:
boltzportfolio??

Signature: boltzportfolio(dataframe, yearly=256, temp=55, floor=0.01, level=0, n=4)
Source:   
def boltzportfolio(dataframe, yearly=256, temp=55, floor=0.01, level=0, n=4):
    '''MAIN: SUMMARY of Boltzmann portfolio, rounded to n-decimal places.
       Return list where computed values are Python floats, not array type, e.g.
           [2.7833,
            [[0.6423, 2.05, 'America'],
             [0.0, -11.17, 'Emerging'],
             [0.0, -10.47, 'Europe'],
             [0.3577, 4.1, 'Gold'],
             [0.0, -4.99, 'Japan']]]
       The portfolio's geometric mean rate is included first.
       Each sub-sublist will consist of weight, rate, and key.
       The order of keys from the dataframe is preserved.
    '''
    rates = gemratarr(dataframe, yearly)
    globalw = weighcovdata(dataframe)
    weights = rentrim(globalw, floor, level)
    pweights = weighsoft(weights, rates, temp, floor, level)
    #      ---- so far should be the same as boltzweigh()
    scores = pweights * rat

In [9]:
#  Part 1 summarized all its computations as follows:
prtf = boltzportfolio( prices[start:end], temp=55, floor=0.01, level=0, n=4 )
prtf

[8.5876,
 [[0.9487, 9.1626, 'America'],
  [0.0, -4.9559, 'Emerging'],
  [0.0, -1.4118, 'Europe'],
  [0.032, -4.0893, 'Gold'],
  [0.0193, 1.335, 'Japan']]]

The first element is the *estimated* geometric mean rate of the entire portfolio.
Respective weights of the underlying assets
are mentioned in the sub-sublist along with the individual geometric mean rates.

## Softmax and temperature

In **reinforcement learning**, the softmax function is used to
convert values into action probabilities.
A positive parameter called the temperature is introduced
to divide through each value. It is a scaling operation such that
high temperatures cause corresponding actions to be equi-probable.
Low temperatures cause a greater difference in selection probability
for actions that differ in their value estimates.
At low temperatures, the probability of the action with the
highest expected reward tends to 1.
The mathematical theory involves cross-entropy which will be
discussed in Part 4.

The **softmax()** function is the essential part of `weighsoft()`,
which is invoked in `boltzportfolio()`,
and many algorithms in machine learning (see References below).

In [10]:
weighsoft??

Signature: weighsoft(weights, rates, temp, floor, level)
Source:   
def weighsoft( weights, rates, temp, floor, level ):
    '''Given weights, compute pweights as array by softmax transform.'''
    scores = weights * rates
    problist = mlearn.softmax( scores, temp )[-1]
    probs = np.array( problist ).reshape(len(problist), 1)
    #  Revise weights based on softmax probabilities:
    pweights = probs * weights
    #  Then appropriately adjust:
    return rentrim(renormalize(pweights), floor, level)
File:      ~/Dropbox/ipy/fecon235/lib/ys_prtf_boltzmann.py
Type:      function


In [11]:
softmax??

Signature: softmax(it, temp=55, n=4)
Source:   
def softmax( it, temp=55, n=4 ):
    '''Softmax probabilities for iterable where temp sets temperature tau.
       Temperature tau is set as a temp percent of ensemble mean so that
       the scaling of tau works well across many different scenarios.
       Experiment with temp around 40 to 70; higher temp (100+)
       will make it-scores more equi-probable, whereas probabilities
       can be sharpened by decreasing temp towards 1.
       Setting temp to 0 results in generic softmax without temperature.
       Results are rounded to n decimal places.
    >>> softmax( [ 16, 8, 4, 0, -8, -16 ], temp=200, n=4 )
    [0, 16, 0.2598, 200, [0.2598, 0.2001, 0.1757, 0.1542, 0.1188, 0.0915]]
    >>> softmax( [ 16, 8, 4, 0, -8, -16 ], temp=50, n=4 )
    [0, 16, 0.5733, 50, [0.5733, 0.2019, 0.1198, 0.0711, 0.0251, 0.0088]]
    >>> softmax( [ 16, 8, 4, 0, -8, -16 ], temp=30, n=4 )
    [0, 16, 0.7773, 30, [0.7773, 0.1365, 0.0572, 0.024, 0.0042, 0.000

## Temperature variations

We vary the scaled temperature `temp`
(proxy for our uncertainty over the estimators)
to see its effect on the Boltzmann portfolio weights,
`pweight` for each agent, and the
geometric mean rate for the portfolio.

In [12]:
#  Show first date of the dataframe:
head(prices, 1)

            America  Emerging  Europe  Gold  Japan
T                                                 
2007-07-25   151.61     46.59   58.96  66.9  58.64

### Vary T where shorts > -10

Note: ***Short positions*** in this notebook are for mathematical illustration.
They ignore limitations imposed by regulations
and the costs introduced by borrowing.
In practice, one would not use the listed securities,
but rather derivatives based on regional indexes to gain *leverage*.

In [13]:
#  Shorts > -10 for 2008-2011

for T in range(1, 302, 20):
    prtf = boltzportfolio(prices['2008':'2011'], temp=T, floor=-10, level=-10, n=2)
    pweights = [ w for w, r, k in prtf[1] ]
    print( T, "\t", prtf[0], "\t", pweights )

1 	 29.84 	 [0.0, -1.0, -0.0, 0.0, 0.0]
21 	 33.18 	 [0.0, -0.89, -0.17, 0.06, 0.0]
41 	 42.86 	 [0.01, -0.94, -0.34, 0.26, 0.01]
61 	 55.37 	 [0.05, -1.11, -0.49, 0.51, 0.03]
81 	 73.37 	 [0.18, -1.41, -0.69, 0.84, 0.08]
101 	 103.81 	 [0.46, -1.97, -1.03, 1.37, 0.17]
121 	 167.68 	 [1.1, -3.18, -1.73, 2.44, 0.37]
141 	 392.2 	 [3.43, -7.47, -4.2, 6.18, 1.06]
161 	 22.84 	 [0.58, -0.32, -0.32, 0.89, 0.17]
181 	 283.42 	 [3.69, -5.51, -3.22, 5.03, 1.01]
201 	 159.95 	 [2.4, -3.14, -1.87, 2.97, 0.64]
221 	 113.85 	 [1.93, -2.26, -1.36, 2.2, 0.49]
241 	 89.63 	 [1.68, -1.8, -1.09, 1.79, 0.42]
261 	 74.96 	 [1.53, -1.52, -0.93, 1.55, 0.37]
281 	 64.91 	 [1.43, -1.33, -0.82, 1.38, 0.34]
301 	 57.74 	 [1.36, -1.2, -0.74, 1.26, 0.32]


If our estimators were exact and held into the future,
at temp=1, the portfolio would be short Emerging markets without leverage.
As the temperature increases, we can see wider diffusion
over the portfolio weights.

The 2008-2011 period was very volatile due to the Great Recession.
At temp=141, the portfolio would be highly leveraged
shorting Emerging markets while also leveraged on the long side with Gold.

***The geometric mean rate of the portfolio is shown by example
not to be a convex function of temperature.***
This means if we want to optimize the portfolio rate,
convex methods can be ruled out.

In [14]:
#  Shorts > -10 for 2011-2014

for T in range(1, 302, 20):
    prtf = boltzportfolio(prices['2011':'2014'], temp=T, floor=-10, level=-10, n=2)
    pweights = [ w for w, r, k in prtf[1] ]
    print( T, "\t", prtf[0], "\t", pweights )

1 	 11.12 	 [1.0, -0.0, -0.0, 0.0, 0.0]
21 	 11.16 	 [1.0, -0.0, -0.0, 0.0, 0.0]
41 	 11.51 	 [1.02, -0.02, -0.02, 0.01, 0.01]
61 	 11.87 	 [1.04, -0.04, -0.05, 0.03, 0.02]
81 	 12.08 	 [1.05, -0.06, -0.08, 0.06, 0.03]
101 	 12.2 	 [1.06, -0.08, -0.1, 0.08, 0.05]
121 	 12.25 	 [1.06, -0.1, -0.13, 0.11, 0.06]
141 	 12.26 	 [1.06, -0.11, -0.14, 0.13, 0.07]
161 	 12.25 	 [1.06, -0.12, -0.16, 0.15, 0.07]
181 	 12.24 	 [1.06, -0.13, -0.17, 0.16, 0.08]
201 	 12.21 	 [1.06, -0.14, -0.18, 0.18, 0.09]
221 	 12.18 	 [1.06, -0.15, -0.19, 0.19, 0.09]
241 	 12.16 	 [1.06, -0.15, -0.2, 0.2, 0.1]
261 	 12.13 	 [1.05, -0.16, -0.21, 0.21, 0.1]
281 	 12.1 	 [1.05, -0.16, -0.21, 0.22, 0.1]
301 	 12.07 	 [1.05, -0.17, -0.22, 0.23, 0.11]


The 2011-2014 period shows that the portfolio rate could perhaps
be a concave function of temperature -- which is false as shown
in the previous period.
In this period, the portfolio is basically long America
at all temperatures.

In [15]:
#  Shorts > -10 for 2014-2017

for T in range(1, 302, 20):
    prtf = boltzportfolio(prices['2014':'2017'], temp=T, floor=-10, level=-10, n=2)
    pweights = [ w for w, r, k in prtf[1] ]
    print( T, "\t", prtf[0], "\t", pweights )

1 	 7.81 	 [1.0, -0.0, -0.0, 0.0, 0.0]
21 	 7.8 	 [1.0, -0.0, -0.0, 0.0, 0.0]
41 	 7.62 	 [0.98, -0.01, -0.0, 0.03, 0.01]
61 	 7.28 	 [0.93, -0.03, -0.0, 0.08, 0.02]
81 	 6.94 	 [0.89, -0.05, -0.01, 0.12, 0.04]
101 	 6.66 	 [0.86, -0.06, -0.01, 0.17, 0.05]
121 	 6.42 	 [0.83, -0.07, -0.01, 0.2, 0.05]
141 	 6.23 	 [0.81, -0.08, -0.01, 0.23, 0.06]
161 	 6.08 	 [0.79, -0.09, -0.01, 0.25, 0.07]
181 	 5.95 	 [0.77, -0.1, -0.01, 0.27, 0.07]
201 	 5.84 	 [0.76, -0.1, -0.01, 0.28, 0.07]
221 	 5.75 	 [0.75, -0.11, -0.01, 0.29, 0.08]
241 	 5.67 	 [0.74, -0.11, -0.01, 0.31, 0.08]
261 	 5.61 	 [0.73, -0.11, -0.01, 0.32, 0.08]
281 	 5.55 	 [0.72, -0.12, -0.01, 0.32, 0.08]
301 	 5.49 	 [0.72, -0.12, -0.01, 0.33, 0.08]


For 2014-2017, the single best asset is America, given temp=1.
More diversification invites Gold into the portfolio.
Variations due to temperature are bland during this period.

### Vary T, for long-only portfolios

We will trim positions less than +0.01 in weight.

In [16]:
#  Long-only for 2008-2011

for T in range(1, 302, 20):
    prtf = boltzportfolio(prices['2008':'2011'], temp=T, floor=0.01, level=0, n=2)
    pweights = [ w for w, r, k in prtf[1] ]
    print( T, "\t", prtf[0], "\t", pweights )

1 	 12.03 	 [0.0, 0.0, 0.0, 1.0, 0.0]
21 	 12.03 	 [0.0, 0.0, 0.0, 1.0, 0.0]
41 	 11.29 	 [0.01, 0.0, 0.0, 0.97, 0.02]
61 	 9.55 	 [0.06, 0.0, 0.0, 0.9, 0.04]
81 	 7.53 	 [0.12, 0.0, 0.0, 0.82, 0.06]
101 	 5.73 	 [0.18, 0.0, 0.0, 0.74, 0.08]
121 	 4.28 	 [0.23, 0.0, 0.0, 0.68, 0.09]
141 	 3.13 	 [0.28, 0.0, 0.0, 0.63, 0.09]
161 	 2.22 	 [0.31, 0.0, 0.0, 0.59, 0.1]
181 	 1.49 	 [0.34, 0.0, 0.0, 0.56, 0.1]
201 	 0.9 	 [0.37, 0.0, 0.0, 0.53, 0.1]
221 	 0.41 	 [0.38, 0.0, 0.0, 0.51, 0.1]
241 	 0.0 	 [0.4, 0.0, 0.0, 0.49, 0.11]
261 	 -0.34 	 [0.42, 0.0, 0.0, 0.48, 0.11]
281 	 -0.63 	 [0.43, 0.0, 0.0, 0.46, 0.11]
301 	 -0.89 	 [0.44, 0.0, 0.0, 0.45, 0.11]


For long-only portfolio, the 2008-2011 period favored Gold over other assets.

***Temperature which is too high, i.e. unnecessary diffusion,
can bring a portfolio's rate of return into negative territory.***

*Meta-question: what if the geometric mean rate of a portfolio
is* **less than the "risk-free" rate of government bonds?**
Maybe we should withdrawn entirely from that portfolio,
or perhaps it is indicating more wealth should be devoted to the portfolio
since the components are becoming cheaply priced.
This traditionally is considered an **asset allocation** problem
where there are different classes of assets such as equities of varying categories,
commodities, real estate, bonds of varying credit risks, and plain cash.

In Part 3, we will let machine learning techniques help us
to answer such meta-questions in the context of learning to make
optimal *sequential* rebalancing decisions when switching costs are included.
The Boltzmann softmax is still at the core of whatever we call
portfolio or asset allocation.

In [17]:
#  Long-only for 2011-2014

for T in range(1, 302, 20):
    prtf = boltzportfolio(prices['2011':'2014'], temp=T, floor=0.01, level=0, n=2)
    pweights = [ w for w, r, k in prtf[1] ]
    print( T, "\t", prtf[0], "\t", pweights )

1 	 11.12 	 [1.0, 0.0, 0.0, 0.0, 0.0]
21 	 11.12 	 [1.0, 0.0, 0.0, 0.0, 0.0]
41 	 10.93 	 [0.99, 0.0, 0.0, 0.01, 0.0]
61 	 10.28 	 [0.95, 0.0, 0.0, 0.03, 0.02]
81 	 9.71 	 [0.91, 0.0, 0.0, 0.06, 0.03]
101 	 9.2 	 [0.88, 0.0, 0.0, 0.08, 0.04]
121 	 8.79 	 [0.86, 0.0, 0.0, 0.09, 0.05]
141 	 8.45 	 [0.84, 0.0, 0.0, 0.11, 0.06]
161 	 8.16 	 [0.82, 0.0, 0.0, 0.12, 0.06]
181 	 7.92 	 [0.8, 0.0, 0.0, 0.13, 0.06]
201 	 7.72 	 [0.79, 0.0, 0.0, 0.14, 0.07]
221 	 7.55 	 [0.78, 0.0, 0.0, 0.15, 0.07]
241 	 7.4 	 [0.77, 0.0, 0.0, 0.15, 0.07]
261 	 7.27 	 [0.77, 0.0, 0.0, 0.16, 0.07]
281 	 7.16 	 [0.76, 0.0, 0.0, 0.16, 0.08]
301 	 7.06 	 [0.75, 0.0, 0.0, 0.17, 0.08]


In [18]:
#  Long-only for 2014-2017

for T in range(1, 302, 20):
    prtf = boltzportfolio(prices['2014':'2017'], temp=T, floor=0.01, level=0, n=2)
    pweights = [ w for w, r, k in prtf[1] ]
    print( T, "\t", prtf[0], "\t", pweights )

1 	 7.81 	 [1.0, 0.0, 0.0, 0.0, 0.0]
21 	 7.81 	 [1.0, 0.0, 0.0, 0.0, 0.0]
41 	 7.58 	 [0.97, 0.0, 0.0, 0.03, 0.0]
61 	 7.05 	 [0.9, 0.0, 0.0, 0.07, 0.02]
81 	 6.61 	 [0.85, 0.0, 0.0, 0.12, 0.03]
101 	 6.24 	 [0.8, 0.0, 0.0, 0.15, 0.04]
121 	 5.96 	 [0.77, 0.0, 0.0, 0.18, 0.05]
141 	 5.73 	 [0.74, 0.0, 0.0, 0.21, 0.06]
161 	 5.55 	 [0.72, 0.0, 0.0, 0.22, 0.06]
181 	 5.4 	 [0.7, 0.0, 0.0, 0.24, 0.06]
201 	 5.27 	 [0.68, 0.0, 0.0, 0.25, 0.07]
221 	 5.17 	 [0.67, 0.0, 0.0, 0.26, 0.07]
241 	 5.08 	 [0.66, 0.0, 0.0, 0.27, 0.07]
261 	 5.01 	 [0.65, 0.0, 0.0, 0.28, 0.07]
281 	 4.94 	 [0.64, 0.0, 0.0, 0.29, 0.07]
301 	 4.88 	 [0.63, 0.0, 0.0, 0.29, 0.07]


When short positions are restricted, assets with
negative geometric mean rates will generally be excluded
from a portfolio.
However, assets with somewhat negative rates
may get included into the portfolio as the
temperature increases.

For the long-only portfolios above, we notice that
the geometric mean rate decreases *monotonically*
as a function of temperature.
This means we should know
how much return we are willing to sacrifice
for possible diversification benefits.

### Is there an ideal temperature?

It may seem subjective, for example,
from the samples so far, that temp=55 is picked as default parameter.
What are the out-of-sample results?

There is no reason why temperature should be fixed over multiple time periods.
In sequential decision situations it may helpful to make the temperature *adaptive*
to maximize terminal wealth.

## Evolution of weights over time at fixed temp

Here we fix `temp` at 55, and see how portfolio weights
evolve over *overlapping* periods.
There will be 100 trading days per period,
and we shall advance 50 days at each step.

In [19]:
#  Short sales PERMITTED but floor is set to -10 and level to -10.

for day in range(100, len(prices), 50):
    prtf = boltzportfolio(prices[day-100:day], temp=55, floor=-10, level=-10, n=2)
    pweights = [ w for w, r, k in prtf[1] ]
    print( day, "\t", prtf[0], "\t", pweights )

100 	 30.77 	 [0.18, -0.08, -0.06, 0.88, 0.08]
150 	 88.05 	 [0.07, -0.34, -0.07, 1.27, 0.07]
200 	 53.75 	 [0.04, -0.98, 0.01, 1.79, 0.14]
250 	 41.15 	 [0.07, -1.33, 0.04, 0.17, 0.06]
300 	 119.14 	 [0.04, -0.85, -0.43, 0.19, 0.06]


/home/yaya/Dropbox/ipy/fecon235/lib/ys_gauss_mix.py:225: RuntimeWarning: invalid value encountered in log
  terms = [ np.log(mu_return), 0, 0, 0, 0 ]


350 	 186.07 	 [0.02, -0.46, -0.65, 0.06, 0.02]
400 	 235.76 	 [0.02, -1.03, -0.39, 0.37, 0.02]
450 	 18.78 	 [0.33, -0.01, -0.06, 0.71, 0.03]
500 	 41.52 	 [0.88, -0.01, -0.02, 0.11, 0.04]
550 	 34.52 	 [0.81, -0.01, -0.01, 0.17, 0.04]
600 	 36.36 	 [0.93, -0.02, -0.02, 0.07, 0.03]
650 	 155.06 	 [3.74, -0.31, -2.83, 0.3, 0.1]
700 	 70.39 	 [1.89, -0.12, -0.84, 0.03, 0.04]
750 	 161.05 	 [0.19, -1.2, -2.36, 4.25, 0.12]
800 	 10.71 	 [0.14, -0.02, -0.02, 0.77, 0.13]
850 	 34.8 	 [0.92, -0.01, -0.02, 0.1, 0.01]
900 	 40.9 	 [1.02, -0.03, -0.01, 0.02, -0.0]
950 	 25.61 	 [0.93, -0.01, -0.01, 0.1, -0.0]
1000 	 40.48 	 [0.1, -0.04, -0.19, 1.12, 0.01]
1050 	 149.63 	 [0.02, -0.96, -0.37, 0.27, 0.05]
1100 	 102.07 	 [0.06, -0.46, -0.78, 0.15, 0.03]
1150 	 41.26 	 [0.93, -0.02, -0.03, 0.05, 0.07]
1200 	 25.85 	 [1.0, -0.02, -0.03, 0.01, 0.04]
1250 	 62.91 	 [0.19, -0.36, -0.92, 0.04, 0.05]
1300 	 11.84 	 [0.94, -0.08, -0.01, 0.15, 0.01]
1350 	 14.49 	 [0.73, -0.01, -0.0, 0.25, 0.04]
1400 	 10

The triple-digit geometric mean rates obviously
cannot be taken at face value (RuntimeWarning issued
but our program works around the mathematical limitation).
They indicate extreme *pathological* conditions in the markets
which will require special handling.
They signal that a regime change is taking place,
and that trading strategies must adapt quickly.
See Appendix 1 for the detailed breakdown,
including a narrative of leptokurtotic events.

Such distress signals will *disappear* when short positions
are restricted, as shown in the next cell,
but the painful consequence is very negative portofolio rates!
In other words, large drawdowns in wealth.

In [20]:
#  Short sales is RESTRICTED because floor is set to 0.01 and level to 0.
#  We want to avoid tiny long positions.

for day in range(100, len(prices), 50):
    prtf = boltzportfolio(prices[day-100:day], temp=55, floor=0.01, level=0, n=2)
    pweights = [ w for w, r, k in prtf[1] ]
    print( day, "\t", prtf[0], "\t", pweights )

100 	 30.64 	 [0.13, 0.0, 0.0, 0.81, 0.05]
150 	 54.14 	 [0.06, 0.0, 0.0, 0.88, 0.06]
200 	 21.32 	 [0.03, 0.0, 0.0, 0.89, 0.08]
250 	 -11.53 	 [0.0, 0.0, 0.12, 0.66, 0.22]
300 	 -22.92 	 [0.01, 0.0, 0.0, 0.87, 0.12]
350 	 -83.95 	 [0.02, 0.0, 0.0, 0.5, 0.49]
400 	 13.67 	 [0.0, 0.0, 0.0, 0.96, 0.04]
450 	 19.38 	 [0.23, 0.0, 0.0, 0.75, 0.02]
500 	 45.61 	 [0.91, 0.0, 0.0, 0.06, 0.03]
550 	 36.19 	 [0.86, 0.0, 0.0, 0.12, 0.03]
600 	 36.56 	 [0.93, 0.0, 0.0, 0.05, 0.02]
650 	 9.3 	 [0.93, 0.0, 0.0, 0.05, 0.01]
700 	 12.62 	 [0.93, 0.0, 0.0, 0.02, 0.04]
750 	 15.66 	 [0.08, 0.0, 0.0, 0.87, 0.06]
800 	 13.64 	 [0.09, 0.0, 0.0, 0.85, 0.06]
850 	 34.62 	 [0.91, 0.0, 0.0, 0.08, 0.01]
900 	 39.86 	 [0.98, 0.0, 0.0, 0.02, 0.0]
950 	 25.54 	 [0.92, 0.0, 0.0, 0.08, 0.0]
1000 	 28.62 	 [0.11, 0.0, 0.0, 0.89, 0.0]
1050 	 22.33 	 [0.01, 0.0, 0.0, 0.9, 0.09]
1100 	 16.79 	 [0.06, 0.0, 0.0, 0.9, 0.03]
1150 	 42.22 	 [0.91, 0.0, 0.0, 0.04, 0.05]
1200 	 24.61 	 [0.95, 0.0, 0.0, 0.01, 0.04]
1250 	 -9.49

For long-only portfolios, Emerging and Europe were rarely included
in the portfolio. It was mostly a switching game between
America and Gold at major turning points.
Such shifts can be seen to occur more gradually if the
periods are more overlapping --
in the way prices are monitored in real-time.

## Closing remarks

We shall cover sequential decisions in another notebook
because portfolio weights alone do not dicate optimal actions.
Here in Part 2 we observed variations in softmax temperature
and the evolution of portfolio weights over time.

Blindly following the weights is not optimal since there
are rebalancing costs to consider.
A long-only portfolio should also monitor long-short signals
to prevent being crushed in extreme market conditions.
See Appendix 1 regarding learning from stress conditions.

Here we considered an equities portfolio but did not
address optimal bet size relative to wealth and how the
information set generated by the Boltzmann weights
can sharpen decisions.

---

## Appendix 1: Portfolio weights under stress conditions

We take a closer look at the global crisis during August 2011
when stock markets globally became highly volatile and leptokurtotic.
The Boltzmann portfolios recorded triple-digit
geometric mean rates by leveraged long and short positions.

The covariance matrix, and hence the pweights, are estimated
within a window of 100 trading days, and advanced one day
to obtain overlapping ("rolling") results.

In [21]:
#  Short sales PERMITTED but floor is set to -10 and level to -10.

for day in range(1000, 1101, 1):
    prtf = boltzportfolio(prices[day-100:day], temp=55, floor=-10, level=-10, n=2)
    pweights = [ w for w, r, k in prtf[1] ]
    print( day, "\t", prtf[0], "\t", pweights )

1000 	 40.48 	 [0.1, -0.04, -0.19, 1.12, 0.01]
1001 	 33.71 	 [0.15, -0.03, -0.13, 1.01, 0.01]
1002 	 30.47 	 [0.2, -0.02, -0.12, 0.94, 0.01]
1003 	 37.78 	 [0.16, -0.03, -0.14, 1.01, 0.01]
1004 	 32.26 	 [0.17, -0.02, -0.14, 0.98, 0.01]
1005 	 33.93 	 [0.15, -0.03, -0.13, 1.01, 0.01]
1006 	 19.03 	 [0.41, -0.01, -0.04, 0.64, 0.0]
1007 	 21.23 	 [0.37, -0.01, -0.05, 0.68, 0.0]
1008 	 32.29 	 [0.19, -0.03, -0.1, 0.94, 0.01]
1009 	 28.76 	 [0.21, -0.03, -0.08, 0.89, 0.01]
1010 	 32.77 	 [0.14, -0.03, -0.15, 1.03, 0.01]
1011 	 38.89 	 [0.1, -0.03, -0.22, 1.14, 0.01]
1012 	 40.02 	 [0.09, -0.03, -0.2, 1.13, 0.01]
1013 	 39.56 	 [0.13, -0.01, -0.18, 1.06, 0.01]
1014 	 62.03 	 [0.07, -0.02, -0.35, 1.28, 0.02]
1015 	 60.1 	 [0.08, -0.02, -0.37, 1.28, 0.04]
1016 	 86.64 	 [0.04, -0.03, -0.5, 1.47, 0.01]
1017 	 52.0 	 [0.06, -0.03, -0.14, 1.1, 0.02]
1018 	 114.82 	 [0.02, -0.08, -0.46, 1.49, 0.02]
1019 	 74.55 	 [0.04, -0.06, -0.21, 1.21, 0.01]
1020 	 133.66 	 [0.02, -0.1, -0.43, 1.49, 0.02]
10

In [22]:
#  Investigate the leptokurtotic prices:
prices[1010:1060]

            America  Emerging  Europe    Gold  Japan
T                                                   
2011-07-28   130.16     46.91   35.89  157.32  42.60
2011-07-29   129.05     47.15   35.90  158.41  42.84
2011-08-01   128.71     47.07   35.02  157.70  42.40
2011-08-02   125.49     45.62   33.63  161.52  42.08
2011-08-03   126.21     45.47   33.97  161.41  42.16
2011-08-04   120.18     42.79   31.36  160.64  39.44
2011-08-05   119.98     42.57   32.47  161.78  39.96
2011-08-08   112.26     39.08   29.48  167.12  38.04
2011-08-09   117.37     41.26   31.31  168.61  40.00
2011-08-10   112.35     39.13   28.98  174.58  38.08
2011-08-11   117.28     41.32   30.36  170.67  39.60
2011-08-12   118.15     41.26   31.30  169.97  39.28
2011-08-15   120.55     42.23   32.24  171.80  39.72
2011-08-16   119.56     41.84   31.52  173.92  39.40
2011-08-17   119.67     42.17   31.62  174.42  39.44
2011-08-18   114.61     40.16   29.91  177.73  38.12
2011-08-19   112.68     39.73   29.17  179.99 

### August 2011 events: 

- ***Black Monday 2011*** refers to 8 August 2011, when stock markets worldwide crashed following the Friday night credit rating downgrade by Standard and Poor's of the **United States sovereign debt** from AAA, or "risk free", to AA+. It was the first time in history the United States was downgraded.

- United States: On 8 August 2011, the S&P 500 lost 79.92 points (6.7%) to 1,119.46 points with all 500 stocks falling.

- On 11 August 2011, the market authorities of Belgium, Italy, France and Spain as well as the European financial regulator ESMA announced the *ban of all forms of short selling on banks and other financial companies* as a result of growing instability in markets on rumours of French banks risking downgrades and concerns of various European banks that are highly exposed to indebted nations such as Greece.

- France: The CAC40 fell by 20% in two weeks. It fell by 5.5% on 18 August 2011. The CAC thus moved from 3,800 points to 3,000 points in 10 days.

- Germany: The DAX fell by 5.8% on 18 August 2011.

Source: https://en.wikipedia.org/wiki/August_2011_stock_markets_fall

- **Physical gold** (not the ETF GLD) at 1623 USD on 1 August 2011, increased to 1878 USD by 22 August 2011.

Source: https://fred.stlouisfed.org/series/GOLDPMGBD228NLBM

The Boltzmann portfolio was mostly long Gold just prior to the crisis,
which in retrospect performed most favorably.
The portfolio towards the end of August 2011 shifts to shorting Emerging.

### Learning from stress conditions

Stress on a system is valuable in understanding
how the components parts can be improved
for sequential decision-making in production.

We shall consider how extreme signals
can be used to generate contrarian decisions in the Part 3.
Excess greed or fear in the markets,
observed as bubbles or anti-bubbles, invite such decisions.

A portfolio manager may seek *less volatile weights*.
This can be achieved in a number of ways:
effectively lengthen the sample size for computing
the covariance matrix, increase softmax temperature,
apply Kalman filtering on the weights themselves
which could be predictive to further help optimize
portfolio returns across multiple rebalancing rules.

## References

- On softmax:
    - https://en.wikipedia.org/wiki/Softmax_function
    - https://compute.quora.com/What-is-softmax
    - http://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative
    - http://cs231n.github.io/linear-classify/#softmax
    - https://en.wikipedia.org/wiki/Reinforcement_learning